In [1]:
import os 
from torch.utils.data import Dataset, DataLoader
import cv2
from torchvision import transforms

In [2]:
class Dess2(Dataset):
    def __init__(self, train_path,mask_path , images_list, masks_list, transform=None):
        self.images_list = images_list
        self.masks_list = masks_list
        self.train_path = train_path
        self.mask_path = mask_path
        self.transform = transform

    def __getitem__(self, index):
        image = cv2.imread(os.path.join(self.train_path, self.images_list[index]))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

        mask = cv2.imread(os.path.join(self.mask_path, self.masks_list[index]), cv2.IMREAD_GRAYSCALE)

        # Apply transformation if specified
        if self.transform is not None:
            transformed = self.transform(image=image, mask=mask)
            image = transformed['image']
            mask = transformed['mask']

        return image, mask,  self.images_list[index]

    def __len__(self):
        return len(self.images_list)
        

class dess400(Dataset):
    def __init__(self, train_path,mask_path , indices, transform=None):
        self.train_path = train_path
        self.mask_path = mask_path
        self.n_samples = len(indices)
        self.transform = transform
        self.indices = indices
        

    def __getitem__(self, index):
        
        name = self.indices[index]

        image = cv2.imread(os.path.join(self.train_path, name))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

        mask = cv2.imread(os.path.join(self.mask_path, name), cv2.IMREAD_GRAYSCALE)

        # Apply transformation if specified
        if self.transform is not None:
            transformed = self.transform(image=image, mask=mask)
            image = transformed['image']
            mask = transformed['mask']

        return image, mask , name

    def __len__(self):
        return self.n_samples

In [3]:


import albumentations as A
from albumentations.pytorch import ToTensorV2
import re


def extract_numerical_part(element):
    match = re.search(r'\d+', element)
    if match:
        return int(match.group())
    else:
        return 0
def round_clip_0_1(x, **kwargs):
    return x.round().clip(0, 1)


train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),
    A.PadIfNeeded(min_height=384, min_width=384, always_apply=True, border_mode=cv2.BORDER_REPLICATE),
    #A.RandomCrop(height=320, width=320, always_apply=True),
    A.GaussNoise(p=0.2),
    A.Perspective(p=0.5),
    A.OneOf(
        [
            A.Sharpen(p=1),
            A.Blur(blur_limit=3, p=1),
            A.MotionBlur(blur_limit=3, p=1),
        ],
        p=0.9,
    ),
    
    ToTensorV2(),  # Convert image and mask to tensors
])
    #A.Lambda(mask=round_clip_0_1),

valid_transform = A.Compose([
    A.Resize(384, 384),
     # Add normalization if required
    ToTensorV2()  # Convert image to tensor
])





In [4]:
#________________________breaking datasets__________________________________make finction asap!!!!

# train_indices=[]
# for x in range(3200):
#         train_indices.append(vs[x][2])
# val_indices=[]

# for x in range(1200):
#         val_indices.append(val[x][2])

#_____save dict indices___ 
# dict_indices = {"train_indices": train_indices, "val_indices": val_indices}
# #save the file 
# with open('dict_indices.json', 'w') as file:
#     json.dump(dict_indices, file)

##domainb_train indices 
# domainb_train=[]
# for x in range(1200):
#         domainb_train.append(domainb_all[x][2])

# domainb_indices = {"train": domainb_train}
# #save the file 
# with open('domainb_indices.json', 'w') as file:
#     json.dump(domainb_indices, file)

In [5]:
# #loading the json file
# import json
# # Load the JSON file
# with open('dict_indices.json', 'r') as file:
#     data = json.load(file)
# train_indices = data['train_indices']
# val_indices = data['val_indices']

# # Load the JSON file
# with open('domainb_indices.json', 'r') as file:
#     data = json.load(file)

# domainb_train = data['train']
# domainb_val = data['val']

In [6]:
# Copyright (c) 2018, Curious AI Ltd. All rights reserved.
#
# This work is licensed under the Creative Commons Attribution-NonCommercial
# 4.0 International License. To view a copy of this license, visit
# http://creativecommons.org/licenses/by-nc/4.0/ or send a letter to
# Creative Commons, PO Box 1866, Mountain View, CA 94042, USA.

"""Functions for ramping hyperparameters up or down

Each function takes the current training step or epoch, and the
ramp length in the same format, and returns a multiplier between
0 and 1.
"""


import numpy as np


def sigmoid_rampup(current, rampup_length):
    """Exponential rampup from https://arxiv.org/abs/1610.02242"""
    if rampup_length == 0:
        return 1.0
    else:
        current = np.clip(current, 0.0, rampup_length)
        phase = 1.0 - current / rampup_length
        return float(np.exp(-5.0 * phase * phase))


def linear_rampup(current, rampup_length):
    """Linear rampup"""
    assert current >= 0 and rampup_length >= 0
    if current >= rampup_length:
        return 1.0
    else:
        return current / rampup_length


def cosine_rampdown(current, rampdown_length):
    """Cosine rampdown from https://arxiv.org/abs/1608.03983"""
    assert 0 <= current <= rampdown_length
    return float(.5 * (np.cos(np.pi * current / rampdown_length) + 1))

def get_current_consistency_weight(epoch):
    # Consistency ramp-up from https://arxiv.org/abs/1610.02242
    return consistency * sigmoid_rampup(epoch, consistency_rampup)


def update_ema_variables(model, ema_model, alpha, global_step):
    # Use the true average until the exponential average is more correct
    alpha = min(1 - 1 / (global_step + 1), alpha)
    for ema_param, param in zip(ema_model.parameters(), model.parameters()):
        ema_param.data.mul_(float(alpha)).add_(1 - float(alpha), param.data)


In [10]:

import random




In [11]:
class Mix(Dataset):
    def __init__(self, train_path, mask_path, testB, test_gtB, mix_images, transform=None):
        self.mix_images = mix_images
        self.train_path = train_path
        self.mask_path = mask_path
        self.b_path = testB
        self.b_mask_path = test_gtB
        
        self.transform = transforms.Compose([
            transforms.Resize((384, 384)),

            transforms.ToTensor(),
            transforms.Normalize(mean=[0.4814, 0.4494, 0.3958],
                                 std=[0.2563, 0.2516, 0.2601])]
        )
        self.noise_transform = transforms.Compose([

            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
            transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
            transforms.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.8, 1.2), shear=10),
            transforms.GaussianBlur(3, sigma=(0.1, 2.0))]
        )
        self.mask_transform = transforms.Compose([
            transforms.Resize(384),

            transforms.PILToTensor()
        ])
        
    def __getitem__(self, index):
        
        full_path = self.mix_images[index]
        image = Image.open(full_path).convert("RGB")
        
    
        image_ema = self.noise_transform(image)
        image = self.transform(image)
        
        if full_path.split("/")[-2] == "train":
            mask = Image.open(os.path.join(self.mask_path, full_path.split("/")[-1])).convert("L")
            mask = self.mask_transform(mask)
            return image, image_ema ,mask,mask
        else:
            mask = Image.open(os.path.join(self.b_mask_path, full_path.split("/")[-1])).convert("L")
            mask = self.mask_transform(mask)
            empty_mask = torch.full((1,384,384), -1, dtype=torch.int64)
            
            return image, image,empty_mask , mask
    

    def __len__(self):
        return len(self.mix_images)

In [12]:
def get_training_augmentation():
    train_transform = [

        A.HorizontalFlip(p=0.5),

        A.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),

        A.PadIfNeeded(min_height=384, min_width=384, always_apply=True, border_mode=cv2.BORDER_REPLICATE),
        #A.RandomCrop(height=320, width=320, always_apply=True),

        A.IAAAdditiveGaussianNoise(p=0.2),
        A.IAAPerspective(p=0.5),

        A.OneOf(
            [
                A.IAASharpen(p=1),
                A.Blur(blur_limit=3, p=1),
                A.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),

        A.Lambda(mask=round_clip_0_1)
    ]
    return A.Compose(train_transform)



In [13]:
class hotencoded(Dataset):
    def __init__(self, train_path, mask_path, images_list, masks_list, num_classes,transform=None):
        self.images_list = images_list
        self.masks_list = masks_list
        self.train_path = train_path
        self.mask_path = mask_path
        self.transform = transform
        self.num_classes = num_classes

    def __getitem__(self, index):
        image = cv2.imread(os.path.join(self.train_path, self.images_list[index]))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

        mask = cv2.imread(os.path.join(self.mask_path, self.masks_list[index]), cv2.IMREAD_GRAYSCALE)

        # Apply transformation if specified
        if self.transform is not None:
            transformed = self.transform(image=image, mask=mask)
            image = transformed['image']
            mask = transformed['mask']

        # Perform one-hot encoding on the mask
        one_hot_mask = torch.zeros((self.num_classes, *mask.shape), dtype=torch.float32)

        # Expand dimensions of `mask` to match the shape of `one_hot_mask`
        expanded_mask = mask.unsqueeze(0)

        # Convert `expanded_mask` to `torch.int64`
        expanded_mask = expanded_mask.to(torch.int64)

        # Use `torch.scatter_` to perform the one-hot encoding
        one_hot_mask = one_hot_mask.scatter_(0, expanded_mask, 1)

        return image, one_hot_mask, mask, self.images_list[index]

    def __len__(self):
        return len(self.images_list)



class hot400(Dataset):
    def __init__(self, train_path,mask_path , indices, num_classes,transform=None):
        self.train_path = train_path
        self.mask_path = mask_path
        self.n_samples = len(indices)
        self.transform = transform
        self.indices = indices
        self.num_classes = num_classes
        

    def __getitem__(self, index):
        
        name = self.indices[index]

        image = cv2.imread(os.path.join(self.train_path, name))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

        mask = cv2.imread(os.path.join(self.mask_path, name), cv2.IMREAD_GRAYSCALE)

        # Apply transformation if specified
        if self.transform is not None:
            transformed = self.transform(image=image, mask=mask)
            image = transformed['image']
            mask = transformed['mask']
        # Perform one-hot encoding on the mask
        one_hot_mask = torch.zeros((self.num_classes, *mask.shape), dtype=torch.float32)

        # Expand dimensions of `mask` to match the shape of `one_hot_mask`
        expanded_mask = mask.unsqueeze(0)

        # Convert `expanded_mask` to `torch.int64`
        expanded_mask = expanded_mask.to(torch.int64)

        # Use `torch.scatter_` to perform the one-hot encoding
        one_hot_mask = one_hot_mask.scatter_(0, expanded_mask, 1)

        return image, one_hot_mask, mask, name

    

    def __len__(self):
        return self.n_samples

In [13]:
# ds = hot400(train_path , mask_path,train_indices,num_classes=7,transform=train_transform)

In [14]:
# # Restore the original mask
# restored_mask = torch.argmax(mask, dim=0)
# orig = orig.to(restored_mask.device)  # Move `orig` tensor to the same device as `restored_mask`

# # Verify if the restored mask is exactly equal to the original mask
# is_equal = torch.allclose(orig, restored_mask)

# # Print the result
# if is_equal:
#     print("The restored mask is exactly equal to the original mask.")
# else:
#     print("The restored mask is not exactly equal to the original mask.")

In [14]:

import albumentations as A
from albumentations.pytorch import ToTensorV2
import re
import json


from torch.utils.data import ConcatDataset


# Load the JSON file
with open('dict_indices.json', 'r') as file:
    data = json.load(file)
train_indices = data['train_indices']
val_indices = data['val_indices']

with open('domainb_indices.json', 'r') as file:
    data = json.load(file)

domainb_indices = data['train']


train_path = os.path.join(os.getcwd(),"train")
mask_path = os.path.join(os.getcwd(),"train_gt")
train_x = os.listdir('train')
train_x.sort(key=extract_numerical_part)
train_y = os.listdir('train_gt')
train_y.sort(key=extract_numerical_part)

val_path = os.path.join(os.getcwd(),"val")
val_gt = os.path.join(os.getcwd(),"val_gt")
val_x = os.listdir('val')
val_x.sort(key=extract_numerical_part)
val_y = os.listdir('val_gt')
val_y.sort(key=extract_numerical_part)

testB = os.path.join(os.getcwd(),"imgs")
test_gtB = os.path.join(os.getcwd(),"Gts")
path_soft_gtB  =  os.path.join(os.getcwd(),"soft_gtB")

testB_images = os.listdir(testB)
testB_images.sort(key=extract_numerical_part)
testB_masks = os.listdir(test_gtB)
testB_masks.sort(key=extract_numerical_part)



vss = dess400(train_path,mask_path,train_indices,train_transform)
vall = dess400(val_path,val_gt,val_indices,train_transform)
dbb = dess400(testB,test_gtB,domainb_indices,train_transform)
domainb_soft = dess400(testB , path_soft_gtB, domainb_indices, train_transform)
# softb = dess400(testB,path_soft_gtB,domainb_train , train_transform)
# testb_val = dess400(testB, test_gtB ,domainb_val,train_transform)

# combined_ds_train = ConcatDataset([vss,softb ])


# train_loader = DataLoader(vss, batch_size=5, shuffle=True)
# val_loader = DataLoader(vall, batch_size=15, shuffle=False)

# mix_loader = DataLoader(combined_ds_train, batch_size=10 , shuffle=True)

vs= Dess2(train_path,mask_path,train_x ,train_y , train_transform)

val= Dess2(val_path , val_gt, val_x,val_y , train_transform)
domainb = hotencoded(testB , test_gtB , testB_images    , testB_masks , num_classes=7,transform=valid_transform)
# ds = hot400(train_path , mask_path,train_indices,num_classes=7,transform=train_transform)
# ds_val = hot400(val_path , val_gt , val_indices,num_classes=7,transform=train_transform)
ds = hotencoded(train_path,mask_path , train_x,train_y , num_classes=7,transform=train_transform)
ds_val = hotencoded(val_path,val_gt,val_x,val_y,num_classes=7,transform=valid_transform)


In [16]:
# data_dir = "/home/khan/Desktop/ammar/MT-UDA-main/data/PnpAda_release_data/train&val/mr_train_tfs"

In [17]:
def splitset(ids,factor):
    split_index = int(len(ids) * factor)
    train= ids[:split_index]
    test = ids[split_index:]
    return train ,test


In [274]:
# IMG_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp')

# class Dataset:
#     def __init__(self, root, transform=None):
#         self.root = root
#         self.imgs = []
#         for dir, _, fnames in sorted(os.walk(root, followlinks=True)):
#             for fname in sorted(fnames):
#                 if fname.lower().endswith(IMG_EXTENSIONS):
#                     self.imgs.append(os.path.join(dir, fname))

#         self.transform = transform if transform is not None else lambda x: x

#     def __getitem__(self, index):
#         with open(self.imgs[index], 'rb') as f:
#             img = Image.open(f)
#             return self.transform(img.convert('RGB'))

#     def __len__(self):
#         return len(self.imgs)